In [1]:
import pandas as pd
import requests

In [53]:
# URI for REST API
url = "https://app.ticketmaster.com/discovery/v2/events?apikey=QOXJm5N0BfIVGZvP2ECxYEGMMm2tvimu&attractionId=K8vZ917KaB7,K8vZ9179DM7,K8vZ9178bS0,K8vZ9175Tr0,K8vZ917Gku7&locale=*"

payload = {}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

In [54]:
response_json = response.json()
events_json = response_json['_embedded']['events']
events = [event_json for event_json in events_json]

In [55]:
event = events[0]
name = event["name"]
id = event["id"]
url = event["url"]
event_start_date = event["dates"]["start"]["localDate"]
event_start_time = event["dates"]["start"]["localTime"]

# subgenre and genre
event_subgenres = [classification["subGenre"]["name"] for classification in event["classifications"]]
event_genres = [classification["genre"]["name"] for classification in event["classifications"]]


priceRanges_min = [priceRange["min"] for priceRange in event["priceRanges"]]
priceRanges_max = [priceRange["max"] for priceRange in event["priceRanges"]]
priceRanges_max
seatmap = event["seatmap"]["staticUrl"]
ageRestrictions = event["ageRestrictions"]["legalAgeEnforced"]
ageRestrictions

venues = event["_embedded"]["venues"]
venue = venues[0]

venue_name = venue["name"]
venue_postalCode = venue["postalCode"]
venue_postalCode
venue_city = venue["city"]["name"]
venue_state_name = venue["state"]["name"]
venue_state_code = venue["state"]["stateCode"]
venue_country = venue["country"]["name"]
venue_country_code = venue["country"]["countryCode"]

venue_address = venue["address"]["line1"]

venue_lat = venue["location"]["latitude"]
venue_long = venue["location"]["longitude"]

attractions = [attraction for attraction in event["_embedded"]["attractions"]]
attraction = attractions[0]

att_name = attraction["name"]




In [56]:
# creating the dataframe
event_df = pd.DataFrame(columns=["id", "name", "url", "start_date", "start_time", "price_min", "price_max", "seatmap_url", "age_restrictions", "city_id"])

genre_df = pd.DataFrame(columns=["id", "sub_genre", "genre", "segment"])

genre_event_bridge_df = pd.DataFrame(columns=["id", "event_id", "sub_genre_id"])

attraction_df = pd.DataFrame(columns=["id", "name"])

event_attraction_df = pd.DataFrame(columns=["id", "attraction_id", "event_id"])

city_df = pd.DataFrame(columns=["id", "city", "state", "state_code", "country", "coountry_code"])


In [58]:
events_list = []
for event in events:

    name = event["name"]
    id = event["id"]
    url = event["url"]
    event_start_date = event["dates"]["start"]["localDate"]
    event_start_time = event["dates"]["start"]["localTime"]
    # TODO: Check price range for multiple values
    priceRanges_min = [priceRange["min"] for priceRange in event["priceRanges"]]
    priceRanges_max = [priceRange["max"] for priceRange in event["priceRanges"]]
    seatmap = event["seatmap"]["staticUrl"]
    ageRestrictions = event["ageRestrictions"]["legalAgeEnforced"]
    # subgenre and genre
    event_subgenres, event_genres, event_segments = [], [], []
    event_subgenres_ids, event_genres_ids, event_segments_ids = [], [], []
    for classification in event["classifications"]:
        event_subgenres.append(classification["subGenre"]["name"])
        event_genres.append(classification["genre"]["name"])
        event_segments.append(classification["segment"]["name"])
        event_subgenres_ids.append(classification["subGenre"]["id"])
        event_genres_ids.append(classification["genre"]["id"])
        event_segments_ids.append(classification["segment"]["id"])
    
    venues = event["_embedded"]["venues"]
    venue_ids, venue_names, venue_postalCodes, venue_citys = [], [], [], []
    venue_state_names, venue_state_codes, venue_countrys, venue_country_codes = [], [], [], []
    venue_addresses, venue_lats, venue_longs = [], [], []
    for venue in venues:
        venue_names.append(venue["name"])
        venue_ids.append(venue["id"])
        venue_postalCodes.append(venue["postalCode"]) 
        venue_citys.append(venue["city"]["name"]) 
        venue_state_names.append(venue["state"]["name"])
        venue_state_codes.append(venue["state"]["stateCode"])
        venue_countrys.append(venue["country"]["name"])
        venue_country_codes.append(venue["country"]["countryCode"]) 

        venue_addresses.append(venue["address"]["line1"])

        venue_lats.append(venue["location"]["latitude"]) 
        venue_longs.append(venue["location"]["longitude"])

    attractions = event["_embedded"]["attractions"]
    artists, artist_ids = [], []
    for attraction in attractions:
        artists.append(attraction["name"])
        artist_ids.append(attraction["id"])

    rec = {
        # event table
        "name" : name, 
        "url" : url, 
        "id" : id ,
        "start_date": event_start_date, 
        "start_time": event_start_time, 
        "price_min": priceRanges_min, 
        "price_max": priceRanges_max, 
        "seatmap_url": seatmap, 
        "age_restrictions": ageRestrictions, 
        
        # genre table
        "event_subgenres": event_subgenres, 
        "event_genres": event_genres, 
        "event_segments": event_segments, 
        "event_subgenres_ids": event_subgenres_ids, 
        "event_genres_ids": event_genres_ids, 
        "event_segments_ids": event_segments_ids,
        
        
        # event table
        "venue_ids": venue_ids, 
        "venue_names": venue_names, 
        "venue_postalCodes": venue_postalCodes, 
        
        # city table
        "venue_citys": venue_citys, 
        "venue_state_names": venue_state_names, 
        "venue_state_codes": venue_state_codes, 
        "venue_countrys": venue_countrys, 
        "venue_country_codes": venue_country_codes, 
        
        # event table
        "venue_addresses": venue_addresses, 
        "venue_lats": venue_lats, 
        "venue_longs": venue_longs,
        
        # artist table columns
        "artists": artists, 
        "artist_ids": artist_ids
    }
    events_list.append(rec)


In [49]:
# getting a list of column names using the dictionary of 1 record
columns = list(rec.keys())
columns

['name',
 'url',
 'id',
 'start_date',
 'start_time',
 'price_min',
 'price_max',
 'seatmap_url',
 'age_restrictions',
 'event_subgenres',
 'event_genres',
 'event_segments',
 'event_subgenres_ids',
 'event_genres_ids',
 'event_segments_ids',
 'venue_ids',
 'venue_names',
 'venue_postalCodes',
 'venue_citys',
 'venue_state_names',
 'venue_state_codes',
 'venue_countrys',
 'venue_country_codes',
 'venue_addresses',
 'venue_lats',
 'venue_longs',
 'artists',
 'artist_ids']

In [59]:
event_v1_df = pd.DataFrame(data = events_list , columns = columns)

In [60]:
event_v1_df

,name,url,id,start_date,start_time,price_min,price_max,seatmap_url,age_restrictions,event_subgenres,...,venue_citys,venue_state_names,venue_state_codes,venue_countrys,venue_country_codes,venue_addresses,venue_lats,venue_longs,artists,artist_ids
0,Offset - Set It Off Tour,https://concerts.livenation.com/offset-set-it-...,vvG1zZbd2170TS,2024-04-10,20:00:00,[59.5],[79.5],https://maps.ticketmaster.com/maps/geometry/3/...,False,[French Rap],...,[Atlanta],[Georgia],[GA],[United States Of America],[US],[800 Battery Ave SE #500],[33.89024],[-84.469717],"[Offset, Skilla Baby, SleazyWorld Go, YRN Mang...","[K8vZ9179DM7, K8vZ917QgL0, K8vZ917QVrV, K8vZ91..."
1,Offset - Set It Off Tour,https://www.ticketmaster.com/offset-set-it-off...,k7vGFbd5I4AHR,2024-03-14,20:00:00,[55.5],[95.5],https://maps.ticketmaster.com/maps/geometry/3/...,True,[French Rap],...,[New York City],[New York],[NY],[United States Of America],[US],[1515 Broadway],[40.757201],[-73.985763],"[Offset, Skilla Baby, SleazyWorld Go, YRN Mang...","[K8vZ9179DM7, K8vZ917QgL0, K8vZ917QVrV, K8vZ91..."
2,Offset - Set It Off Tour,https://concerts.livenation.com/offset-set-it-...,vv1AvZkZwGkeY9NrU,2024-03-15,19:00:00,[59.5],[79.5],https://maps.ticketmaster.com/maps/geometry/3/...,False,[French Rap],...,[Boston],[Massachusetts],[MA],[United States Of America],[US],[15 Lansdowne St.],[42.3471841],[-71.0957036],"[Offset, Skilla Baby, SleazyWorld Go, YRN Mang...","[K8vZ9179DM7, K8vZ917QgL0, K8vZ917QVrV, K8vZ91..."
3,Offset - Set It Off Tour,https://concerts.livenation.com/offset-set-it-...,1A4ZkZwGkdJYwWn,2024-03-12,20:00:00,[59.5],[79.5],https://maps.ticketmaster.com/maps/geometry/3/...,False,[French Rap],...,[Silver Spring],[Maryland],[MD],[United States Of America],[US],[8656 Colesville Road],[38.99734055],[-77.02746183],"[Offset, Skilla Baby, SleazyWorld Go, YRN Mang...","[K8vZ9179DM7, K8vZ917QgL0, K8vZ917QVrV, K8vZ91..."
4,Offset - Set It Off Tour,https://concerts.livenation.com/offset-set-it-...,vv1AeZkZSGkeXw4Pr,2024-03-10,20:00:00,[59.5],[79.5],https://maps.ticketmaster.com/maps/geometry/3/...,False,[French Rap],...,[Philadelphia],[Pennsylvania],[PA],[United States Of America],[US],[29 E Allen Street],[39.965803],[-75.134686],"[Offset, Skilla Baby, SleazyWorld Go, YRN Mang...","[K8vZ9179DM7, K8vZ917QgL0, K8vZ917QVrV, K8vZ91..."
5,Offset - Set It Off Tour,https://concerts.livenation.com/offset-set-it-...,G5vYZbdC9Ed-8,2024-03-30,20:00:00,[45.5],[75.5],https://maps.ticketmaster.com/maps/geometry/3/...,False,[French Rap],...,[San Francisco],[California],[CA],[United States Of America],[US],[1111 California St],[37.791419],[-122.413293],"[Offset, Skilla Baby, SleazyWorld Go, YRN Mang...","[K8vZ9179DM7, K8vZ917QgL0, K8vZ917QVrV, K8vZ91..."
6,Offset - Set It Off Tour,https://concerts.livenation.com/offset-set-it-...,vv1AFZkZwGketOwxC,2024-03-19,19:00:00,[37.5],[77.5],https://maps.ticketmaster.com/maps/geometry/3/...,False,[French Rap],...,[Detroit],[Michigan],[MI],[United States Of America],[US],[2115 Woodward (Next To the Fox)],[42.4605914],[-83.2129839],"[Offset, Skilla Baby, SleazyWorld Go, YRN Mang...","[K8vZ9179DM7, K8vZ917QgL0, K8vZ917QVrV, K8vZ91..."
7,Offset - Set It Off Tour,https://concerts.livenation.com/offset-set-it-...,vv1AaZkZSGkeiOg8_,2024-03-29,19:00:00,[59.5],[79.5],https://maps.ticketmaster.com/maps/geometry/3/...,False,[French Rap],...,[Hollywood],[California],[CA],[United States Of America],[US],[6215 Sunset Blvd],[34.09850537],[-118.32421313],"[Offset, Skilla Baby, YRN Mango Foo, SleazyWor...","[K8vZ9179DM7, K8vZ917QgL0, K8vZ917jbtf, K8vZ91..."
8,Offset - Set It Off Tour,https://concerts.livenation.com/offset-set-it-...,1avbZbd89k4ZdGv1,2024-03-22,20:00:00,[59.5],[79.5],https://maps.ticketmaster.com/maps/geometry/3/...,False,[French Rap],...,[Cincinnati],[Ohio],[OH],[United States Of America],[US],[2621 Short Vine Street],[39.129949],[-84.508882],"[Offset, Skilla Baby, SleazyWorld Go, YRN Mang...","[K8vZ9179DM7, K8vZ917QgL0, K8vZ917QVrV, K8vZ91..."
9,Offset - Set It 

In [ ]:
import os
from datetime import datetime

import pandas as pd
import pytz
import requests

import config

year_to_fetch = 2022
url_events_amsterdam_center_2022 = 'https://app.ticketmaster.com/discovery/v2/events.json?size=200&geoPoint=u173z&radius=5&startDateTime={year}-01-01T00:00:00Z&endDateTime={year}-12-30T00:00:00Z&unit=km&sort=distance,date,asc&apikey={api_key}'.format(
    year=year_to_fetch, api_key=config.api_key)
output_csv = 'events_amsterdam_center_{}_{}_UTC.csv'.format(year_to_fetch,
                                                            datetime.now(pytz.utc).strftime("%d-%m-%Y_%H.%M.%S"))


class Event:
    def __init__(self, event_json):
        self.name = event_json['name']
        if event_json['_embedded']['venues'][0]['address']['line1'] == 'Lijnbaansgracht 238':
            self.venue = 'Lovelee'
        elif event_json['_embedded']['venues'][0]['address']['line1'] == 'Lijnbaansgracht 234 A':
            self.venue = 'Melkweg'
        else:
            self.venue = event_json['_embedded']['venues'][0]['name']
        self.datetime = event_json['dates']['start']['dateTime']

    def __str__(self):
        return 'Date: {}\nName: {}\nVenue: {}'.format(self.datetime, self.name,
                                                      self.venue)


def get_events_amsterdam_center_2022_page(page_id):
    return url_events_amsterdam_center_2022 + '&page=' + str(page_id)


def get_response_json(url):
    response = requests.get(url)
    response_json = response.json()

    if 'errors' in response_json:  # error encountered or last page reached
        return None

    return response_json


def main():
    response_json = get_response_json(get_events_amsterdam_center_2022_page(0))
    if response_json is None:
        print('Error encountered. Stopping execution...')
        exit()

    total_pages = response_json['page']['totalPages']

    write_header = not os.path.exists(output_csv) or (os.path.exists(output_csv) and os.stat(output_csv).st_size == 0)
    events_df = None
    for page_id in range(total_pages):
        response_json = get_response_json(get_events_amsterdam_center_2022_page(0))
        if response_json is None:
            print('Error encountered. Stopping execution...')
            exit()

        events_json = response_json['_embedded']['events']

        events = [Event(event_json) for event_json in events_json if len(event_json['_embedded']['venues']) > 0]

        if events_df is None:
            events_df = pd.DataFrame(columns=events[0].__dict__.keys())

        for event in events:
            events_df = events_df.append(event.__dict__, ignore_index=True)

    if events_df is not None:
        events_df.to_csv(output_csv, mode='a', header=write_header)


if __name__ == '__main__':
    main()